<a href="https://colab.research.google.com/github/sksos7/Project/blob/main/Codestates_AI_05/Project_S_01/Project_S1_kyu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goal of project : "다음 분기에 어떤 게임을 설계해야 할까?"

게임 회사의 데이터 팀

데이터셋을 읽는 것 부터, 이러한 게임을 만들어야 한다는 결론까지 의 과정

해당 내용을 기반으로 설명하는 영상을 작성

발표를 듣는 사람은 비데이터 직군이라 가정

Sprint 1 : EDA, Feature Engineering, Data Manipulation, Data Visualization

Sprint 2 : Hypothesis Test, CI, Bayesian

Sprint 3 : DR, PCA, Cluster

1. 지역에 따라서 선호하는 게임 장르가 다를까 라는 질문에 대답을 하셔야합니다.
2. 연도별 게임의 트렌드가 있을까 라는 질문에 대답을 하셔야합니다.
3. 출고량이 높은 게임에 대한 분석 및 시각화 프로세스가 포함되어야 합니다.

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
"""
Name : 게임의 이름입니다.
Platform : 게임이 지원되는 플랫폼의 이름입니다.
Year : 게임이 출시된 연도입니다.
Genre : 게임의 장르입니다.
Publisher : 게임을 제작한 회사입니다.
NA_Sales : 북미지역에서의 출고량입니다.
EU_Sales : 유럽지역에서의 출고량입니다.
JP_Sales : 일본지역에서의 출고량입니다.
Other_Sales : 기타지역에서의 출고량입니다.
"""
# 데이터셋 불러오기
df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/vgames2.csv')

# (16598, 10)
df.head()

In [ ]:
# 결측치 확인
df.isna().sum()

In [ ]:
# 결측치의 모든 값을 'Unknown' 으로 대체
df = df.fillna('Unknown')

In [ ]:
# 출고량을 변환하는 함수
def change_shipment_volume(df,colums):
    count = 0
    for i in df[colums]:
        if 'K' in i:
            tmp = float(i.replace('K',''))
            if tmp != 0:
                df.loc[count,colums] = tmp ** -3
            else : df.loc[count,colums] = tmp
        elif 'M' in i:
            tmp = float(i.replace('M',''))
            df.loc[count,colums] = tmp * 1
        count = count + 1
    return df[colums]

In [ ]:
# 모든 출고량 'M' 지우고 x1, 'K' 지우고 **-3
df['NA_Sales'] = pd.to_numeric(change_shipment_volume(df,'NA_Sales'))
df['EU_Sales'] = pd.to_numeric(change_shipment_volume(df,'EU_Sales'))
df['JP_Sales'] = pd.to_numeric(change_shipment_volume(df,'JP_Sales'))
df['Other_Sales'] = pd.to_numeric(change_shipment_volume(df,'Other_Sales'))

In [ ]:
# 날짜를 적절하게 바꿈
count = 0
for i in df['Year']:
    if i == 'Unknown':
        count = count + 1
        continue
    elif i < 25:
        df.loc[count,'Year'] = i + 2000
    elif (i > 25) and (i < 100):
        df.loc[count,'Year'] = i + 1900
    count = count + 1

In [ ]:
# 가장 최근에 출시된 연도는 2020년
df[df['Year']==2020]

In [ ]:
df = df.drop('Unnamed: 0',axis=1)

In [ ]:
df.head()



>   위의 과정은 전처리 과정










> 1. 지역에 따라서 선호하는 게임 장르가 다를까 ?



In [ ]:
# 필요한 부분만 가져오기
# 장르 + 각 지역별 출고량
df_area_genre = df.copy()
df_area_genre = df_area_genre.loc[:,['Genre','NA_Sales','EU_Sales','JP_Sales','Other_Sales']]
# 장르가 'Unknown' 인 데이터는 제외
df_area_genre = df_area_genre[~(df_area_genre['Genre'] == 'Unknown')]
df_area_genre.reset_index(drop=True, inplace=True)
df_area_genre

In [ ]:
# 장르를 기준으로 그룹화
df_area_genre_visual = df_area_genre.groupby('Genre').sum()
df_area_genre_visual

In [ ]:
# 지역별 각 장르의 출고량
df_area_genre_visual.plot.bar()
plt.show()

In [ ]:
# 지역별 action, role-playing 출고량
df_area_genre_visual.loc[['Action','Role-Playing'],:].plot.bar()
plt.show()

In [ ]:
# 지역별 action, role-playing 출고량의 합
df_area_genre_visual.loc[['Action','Role-Playing'],:].plot.bar(stacked = True)
plt.show()

In [ ]:
# 북미 지역의 장르 출고량
plt.figure(figsize=(10,10))

explode = [0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.05, 0.0]
wedgeprops={'width': 0.9, 'edgecolor': 'w', 'linewidth': 1}
plt.pie(df_area_genre_visual['NA_Sales'],
        labels = df_area_genre_visual.index,
        explode = explode,
        autopct='%1.1f%%',
        textprops = {'fontsize':13},
        wedgeprops=wedgeprops)



plt.title('NA', fontdict={ 'fontsize': 25, 'fontweight': 'bold'})
plt.show()

In [ ]:
# 유럽 지역의 장르 출고량
plt.figure(figsize=(10,10))
explode = [0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.05, 0.0]
plt.pie(df_area_genre_visual['EU_Sales'],
        labels = df_area_genre_visual.index,
        explode = explode,
        autopct='%1.1f%%',
        textprops = {'fontsize':13},
        wedgeprops=wedgeprops)

plt.title('EU', fontdict={ 'fontsize': 25, 'fontweight': 'bold'})
plt.show()

In [ ]:
# 일본 지역의 장르 출고량
plt.figure(figsize=(10,10))
explode = [0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.05, 0.0]
plt.pie(df_area_genre_visual['JP_Sales'],
        labels = df_area_genre_visual.index,
        explode = explode,
        autopct='%1.1f%%',
        textprops = {'fontsize':13},
        wedgeprops=wedgeprops)

plt.title('JP', fontdict={ 'fontsize': 25, 'fontweight': 'bold'})
plt.show()

In [ ]:
# 그외 지역의 장르 출고량
plt.figure(figsize=(10,10))
explode = [0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.05, 0.0]
plt.pie(df_area_genre_visual['Other_Sales'],
        labels = df_area_genre_visual.index,
        explode = explode,
        autopct='%1.1f%%',
        textprops = {'fontsize':13},
        wedgeprops=wedgeprops)

plt.title('Other', fontdict={ 'fontsize': 25, 'fontweight': 'bold'})
plt.show()


> 2. 연도별 게임의 트렌드가 있을까 라는 질문에 대답을 하셔야합니다.




In [ ]:
# 필요한 부분만 가져오기
# 플랫폼 + 연도 + 장르
df_year_trend = df.copy()
df_year_trend = df_year_trend.loc[:,['Platform','Year', 'Genre']]
# 연도와 장르가 'Unknown' 인 값을 제외한 데이터
condition = (df_year_trend['Year']=='Unknown') | (df_year_trend['Genre']=='Unknown')
df_year_trend = df_year_trend[~condition]
df_year_trend.reset_index(drop=True, inplace=True)
df_year_trend.head()

In [ ]:
# 먼저 연도와 장르
df_year_genre = df_year_trend.loc[:,['Year','Genre']].copy()
df_year_genre.head()

In [ ]:
# 연도와 장르의 빈도수
df_year_genre_visual = pd.crosstab(df_year_genre['Year'],df_year_genre['Genre'])
df_year_genre_visual.head()

In [ ]:
# 연도별 장르의 합 (출시한 게임 수)
df_year_genre_visual.plot(kind ='bar', figsize=(25,10), fontsize=15, stacked = True)
plt.show()

In [ ]:
# 연도별 장르
df_year_genre_visual.plot(kind ='bar', figsize=(25,10), fontsize=15)
plt.show()

In [ ]:
# 연도별 장르 ( 액션, 스포츠, 슈터, 롤플레잉)
df_year_genre_visual.loc[:,['Action','Sports','Shooter','Role-Playing']].plot(kind ='bar', figsize=(25,10), fontsize=15)
plt.show()

In [ ]:
# 필요한 데이터만 불러오기
# 플렛폼의 비율
df_name_platform = df.loc[:,['Name','Platform']]
df_name_platform.head()

In [ ]:
df_name_platform_visual = df_name_platform.groupby(['Platform']).count()
df_name_platform_visual = df_name_platform_visual.sort_values(by='Name' ,ascending=False)

plt.figure(figsize=(15,10))
colors = sns.color_palette('hls',len(df_name_platform_visual['Name']))
plt.bar(df_name_platform_visual.index, df_name_platform_visual['Name'], color = colors)
plt.show()

In [ ]:
# 필요한 데이터만 불러오기
# 년도 별 플랫폼
df_year_platform = df_year_trend.loc[:,['Year','Platform']].copy()
df_year_platform.head()

In [ ]:
# 플랫폼 연도의 빈도수
df_year_platform_visual = pd.crosstab(df_year_platform['Platform'],df_year_platform['Year'])
df_year_platform_visual.columns = df_year_platform_visual.columns.astype(int)
df_year_platform_visual.head()

In [ ]:
# 시각화 1980~1989
plt.figure(figsize=(25,10))

title_font = {
    'fontsize': 20,
    'fontweight': 'bold'
}

for i in range(0,10):
    plt.subplot(2,5,i+1)
    condition = df_year_platform_visual.iloc[:,i] == 0
    plt.pie(df_year_platform_visual[~condition].iloc[:,i],
            labels = df_year_platform_visual[~condition].iloc[:,0].index,
            autopct='%.1f%%')
    plt.title(df_year_platform_visual.columns[i], fontdict=title_font)

plt.show()

In [ ]:
# 시각화 1990~1999
plt.figure(figsize=(25,10))

for i in range(10,20):
    plt.subplot(2,5,i+1-10)
    condition = df_year_platform_visual.iloc[:,i] == 0
    plt.pie(df_year_platform_visual[~condition].iloc[:,i],
            labels = df_year_platform_visual[~condition].iloc[:,0].index,
            autopct='%.1f%%')
    plt.title(df_year_platform_visual.columns[i], fontdict=title_font)

plt.show()

In [ ]:
# 시각화 2000 ~ 2009
plt.figure(figsize=(25,10))

for i in range(20,30):
    plt.subplot(2,5,i+1-20)
    condition = df_year_platform_visual.iloc[:,i] == 0
    plt.pie(df_year_platform_visual[~condition].iloc[:,i],
            labels = df_year_platform_visual[~condition].iloc[:,0].index,
            autopct='%.1f%%')
    plt.title(df_year_platform_visual.columns[i], fontdict=title_font)

plt.show()

In [ ]:
# 2010 ~ 2020 (17 18 19 20 은 데이터가 적어 제외)
plt.figure(figsize=(25,10))

for i in range(30,39):
    plt.subplot(2,5,i+1-30)
    condition = df_year_platform_visual.iloc[:,i] == 0
    plt.pie(df_year_platform_visual[~condition].iloc[:,i],
            labels = df_year_platform_visual[~condition].iloc[:,0].index,
            autopct='%.1f%%')
    plt.title(df_year_platform_visual.columns[i], fontdict=title_font)

plt.show()

In [ ]:
# 필요한 것만 빼오기
# 플렛폼과 각 지역의 출고량
df_platform_area = df.loc[:,['Platform','NA_Sales','EU_Sales','JP_Sales','Other_Sales']].copy()
df_platform_area.head()

In [ ]:
# 출고량의 총합에 대해 플랫폼으로 그룹화
df_platform_area['Platform_Total'] = df_platform_area['NA_Sales'] + df_platform_area['EU_Sales'] + df_platform_area['JP_Sales'] + df_platform_area['Other_Sales']
df_platform_area_visual = df_platform_area.loc[:,['Platform','Platform_Total']].groupby(['Platform']).sum()
df_platform_area_visual = df_platform_area_visual.sort_values(by='Platform_Total' ,ascending=False)
df_platform_area_visual.head()

In [ ]:
# 각 플랫폼의 출고량
plt.figure(figsize=(15,10))
colors = sns.color_palette('hls',len(df_platform_area_visual['Platform_Total']))
plt.bar(df_platform_area_visual.index, df_platform_area_visual['Platform_Total'], color = colors)
plt.show()

In [ ]:
# 2016년 기준 출시된 플랫폼 게임의 출고량
condition = df_platform_area_visual.loc[['X360','PS3','PS4','PC','3DS','XOne','WiiU','PSV'],:]
plt.bar(condition.index, condition['Platform_Total'], color = colors)
plt.show()



> 3. 출고량이 높은 게임에 대한 분석 및 시각화 프로세스가 포함



In [ ]:
# 출고량(각 지역별 출고량의 합) + 게임 일름
# 필요한 것만 가져오기
df_name_area = df.loc[:,['Name','NA_Sales','EU_Sales','JP_Sales','Other_Sales']].copy()
df_name_area['Name_Total'] =  df_name_area['NA_Sales'] + df_name_area['EU_Sales'] + df_name_area['JP_Sales'] + df_name_area['Other_Sales']
df_name_area.head()

In [ ]:
# 각 게임별 출고량, 다른 플렛폼일 경우 합쳐서 계산
df_name_area_visual = df_name_area.loc[:,['Name','Name_Total']].groupby(['Name']).sum()
df_name_area_visual = df_name_area_visual.sort_values(by='Name_Total' ,ascending=False)
df_name_area_visual.head()

In [ ]:
# 각 게임별 출고량 시각화 (약 11000개의 데이터가 있어서 상위 100개)
condition = df_name_area_visual.iloc[0:50,:]
plt.figure(figsize=(20,10))
colors = sns.color_palette('hls',len(condition['Name_Total']))
plt.barh(condition.index, condition['Name_Total'], color = colors)
plt.show()

In [ ]:
# 출고량(각 지역별 출고량의 합) + 게임 + 장르
# 필요한 것만 가져오기
df_name_genre_area = df.loc[:,['Name','Genre','NA_Sales','EU_Sales','JP_Sales','Other_Sales']].copy()
df_name_genre_area['Name_Total'] =  (df_name_genre_area['NA_Sales'] +
                                     df_name_genre_area['EU_Sales'] +
                                     df_name_genre_area['JP_Sales'] +
                                     df_name_genre_area['Other_Sales'])
df_name_genre_area.head()

In [ ]:
# 가장 많은 출고량의 데이터 50개를 추출
df_name_genre_area_to_extract = df_name_genre_area.loc[:,['Name','Genre','Name_Total']]
df_name_genre_area_to_extract = df_name_genre_area_to_extract.groupby(['Name','Genre']).sum()
df_name_genre_area_to_extract = df_name_genre_area_to_extract.sort_values(by='Name_Total' ,ascending=False)
df_name_genre_area_visual = df_name_genre_area_to_extract.iloc[0:50,:]
df_name_genre_area_visual.head()

In [ ]:
# 출고량이 제일 높은 50개를 기준으로 장르의 퍼센테이지 계산
df_name_genre_area_visual = df_name_genre_area_visual.reset_index()
df_name_genre_area_visual = df_name_genre_area_visual.drop('Name',axis=1)
df_name_genre_area_visual = df_name_genre_area_visual.groupby(['Genre']).count()

plt.figure(figsize=(25,10))
# np.array(values).ravel()
plt.pie(df_name_genre_area_visual['Name_Total'],
        labels = df_name_genre_area_visual.index,
        autopct='%.1f%%')
plt.title('Top 50 Genre Percentage', fontdict=title_font)
plt.show()

In [ ]:
# 출고량이 높은 50, 100, 150, 200개의 장르 퍼센트
plt.figure(figsize=(15,15))

title_font = {
    'fontsize': 15,
    'fontweight': 'bold'
}

for i in range(1,5):
    df_name_genre_area_visual = df_name_genre_area_to_extract.iloc[0:50*i,:]
    df_name_genre_area_visual = df_name_genre_area_visual.reset_index()
    df_name_genre_area_visual = df_name_genre_area_visual.drop('Name',axis=1)
    df_name_genre_area_visual = df_name_genre_area_visual.groupby(['Genre']).count()

    plt.subplot(2,2,i)
    condition = df_year_platform_visual.iloc[:,i] == 0
    plt.pie(df_name_genre_area_visual['Name_Total'],
            labels = df_name_genre_area_visual.index,
            autopct='%.1f%%')
    plt.title( ('Top '+str(50*i)+' Genre Percentage') , fontdict=title_font)
plt.show()